In [ ]:
import torch
# import torchvision
# from torch.utils.data import DataLoader, Dataset
# import pytorch_lightning as pl
import pandas as pd
# from dataLoader.ShadowDataset import ShadowData, ShadowDatasetTest, ShadowDataset
# from transforms.shadowTransforms import ShadowTrainTransforms, shadowValTransforms
# from myModels import ResNetModel, MYEfficientNetModel #,EfficientNetModel
# from pl_bolts.transforms.dataset_normalizations import(
#     imagenet_normalization
# )
# from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning.strategies.ddp import DDPStrategy

# from pytorch_lightning.loggers import NeptuneLogger, TensorBoardLogger
# import pickle
# from pytorch_lightning.profiler import PyTorchProfiler
# from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import tqdm
from captum.attr import GuidedGradCam

In [ ]:
from myModelsAll import MyEfficientNet_3channel
from config import MyTansforms,MyDataset
from data_loader import MyDataLoading_color,MyDataLoading_four_channel

In [ ]:
myModelPath = "/home/atharva/Abha/Shadows/blender_pipeline/pipeline1/outputModels/model-epoch=09-val_loss=0.35.ckpt"
base_path = "/panfs/jay/groups/27/kersten/gejji003/Shadows/data/images_rendered_try2/" # loaction of train test
file_path_test = base_path +"testData.csv"
file_path_train = base_path +"trainData.csv"
file_path_val = base_path +"valData.csv"

train_df = pd.read_csv(file_path_train)
val_df = pd.read_csv(file_path_val)
test_df = pd.read_csv(file_path_test)

In [ ]:
theDataLoader = MyDataLoading_color
color_channel = "colors"
shadow_channel = "cast_shadow"

In [ ]:

model = MyEfficientNet_3channel.load_from_checkpoint(myModelPath)
model.eval()
model.cuda()

dataset = MyDataset(train_df,val_df,test_df,color_channel,shadow_channel,theDataLoader)
test_loader = dataset.test_dataloader()


In [ ]:
last_conv_layer = None
myGradcam = GuidedGradCam(model, last_conv_layer)
y_all = []
pred_all = []
with torch.no_grad():
    pbar = tqdm(enumerate(test_loader), total=len(test_loader))
    for idx, X in pbar:
        X, y = X
        X = X.cuda().contiguous()
        # print("TYPE X: ", type(X))
        # y = y.cuda()
        myPred = model(X).squeeze().cuda()
        myPred = myPred.cpu()
        print(myPred)
        # print(myPred)
        # if myPred > 0.08:
        #     myPred = 1
        # else:
        #     myPred = 0
        myPred = np.argmax(myPred)
        myAttr = myGradcam.attribute(X, myPred)           
        # print(type(myPred))
        # y = y.cuda()
        # print(type(y))
        # print(y)
        y_all.append(y.int().numpy())
        # print(myPred)
        pred_all.append(myPred)
    myClassRep = classification_report(y_all, pred_all)
    print(myClassRep)
    myConfMat = confusion_matrix(y_all, pred_all)
    print(myConfMat)


In [ ]:
def main(myModelPath, myCSVPath, myBatchSize, myNumWorkers):
    model = MYEfficientNetModel.load_from_checkpoint(myModelPath)
    model.eval()
    model.cuda()

    df_test = pd.read_csv(myCSVPath)
# df=self.dfTest, mount_point=self.mountPoint, label=self.label, img_col=self.img_path, color=self.color, transform=self.valid_transform
    test_ds = ShadowDataset(df=df_test, img_col="path_to_img", label="label", color="colors", shadowMask= "cast_shadow", mount_point="./", depth="depth", transform=shadowValTransforms)

    test_loader = DataLoader(test_ds, batch_size=myBatchSize, shuffle=False, num_workers=myNumWorkers, pin_memory=True, prefetch_factor=4)
    last_conv_layer = None
    myGradcam = GuidedGradCam(model, last_conv_layer)
    y_all = []
    pred_all = []
    with torch.no_grad():
        pbar = tqdm(enumerate(test_loader), total=len(test_loader))
        for idx, X in pbar:
            X, y = X
            X = X.cuda().contiguous()
            # print("TYPE X: ", type(X))
            # y = y.cuda()
            myPred = model(X).squeeze().cuda()
            myPred = myPred.cpu()
            print(myPred)
            # print(myPred)
            # if myPred > 0.08:
            #     myPred = 1
            # else:
            #     myPred = 0
            myPred = np.argmax(myPred)
            myAttr = myGradcam.attribute(X, myPred)           
            # print(type(myPred))
            # y = y.cuda()
            # print(type(y))
            # print(y)
            y_all.append(y.int().numpy())
            # print(myPred)
            pred_all.append(myPred)
            # print(y_all)
            # print(pred_all)
        
        myClassRep = classification_report(y_all, pred_all)
        print(myClassRep)
        myConfMat = confusion_matrix(y_all, pred_all)
        print(myConfMat)


if __name__ == "__main__":
    # myModelPath = "/home/atharva/Abha/Shadows/blender_pipeline/pipeline1/outputModels/epoch=3-val_loss=0.6943.ckpt"
    # myModelPath = "/home/atharva/Abha/Shadows/blender_pipeline/pipeline1/outputModels/model-epoch=11-val_loss=0.34.ckpt"
    myModelPath = "/home/atharva/Abha/Shadows/blender_pipeline/pipeline1/outputModels/model-epoch=09-val_loss=0.35.ckpt"
    myCSVPath = "/home/atharva/Abha/Shadows/blender_pipeline/pipeline1/dataFiles/testData.csv"
    myBatchSize = 1
    myNumWorkers = 4
    main(myModelPath, myCSVPath, myBatchSize, myNumWorkers)

